In [ ]:
from sedona.spark import *
config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

In [ ]:
database = 'gde_gold'
sedona.sql(f'CREATE DATABASE IF NOT EXISTS org_catalog.{database}')

In [ ]:
df = sedona.sql(f'''
select *,
ST_GeoHash(geometry, 15) AS geohash,
struct(st_xmin(geometry) as xmin, st_ymin(geometry) as ymin, st_xmax(geometry) as xmax, st_ymax(geometry) as ymax) as bbox
from org_catalog.{database}.analytics_gold
''')

In [ ]:
%%time
import os

user_uri = os.getenv("USER_S3_PATH")

df.repartitionByRange(1, "geohash") \
    .sortWithinPartitions("geohash") \
    .drop("geohash15") \
    .write \
    .format("geoparquet") \
    .option("geoparquet.version", "1.1.0") \
    .option("geoparquet.covering", "bbox") \
    .save(user_uri + "gde_geoparquet", mode='overwrite', compression='snappy')